# TfIdf + first attempts of fitting Naive Bayes & SVC

In [1]:
# importing packages

# heavy
from sklearn.feature_extraction.text import TfidfVectorizer




In [11]:
# for recommendations
from sklearn.metrics.pairwise import linear_kernel

# ml models
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm

# ml stuff
from sklearn.model_selection import train_test_split
# other
import pandas as pd
import numpy as np


In [3]:
# loading data
aita = pd.read_csv('../data/aita.csv', index_col=0)



C:\Users\Patryk\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Learning Naive Bayes

In [20]:
nb = MultinomialNB()

In [69]:
aita.head()

,flair,title,text,score,url,num_comments,created_utc,is_self,num_crossposts,title_wc,selftext_wc,title_cc,selftext_cc
14,asshole,aita for missing my shift at work and studying?,i'm a highschool senior who works a part time ...,16.0,http://www.reddit.com/r/AmItheAsshole/comments...,17.0,1.422047e+09,True,NaN,9,120,47,614
18,asshole,aita for using robin williams' suicide to brea...,on okcupid this girl listed only robin william...,13.0,http://www.reddit.com/r/AmItheAsshole/comments...,12.0,1.422843e+09,True,NaN,10,69,56,395
27,asshole,aita for creating a fucked up love triangle?,so it started last semester as i'm a third yea...,10.0,http://www.reddit.com/r/AmItheAsshole/comments...,16.0,1.423202e+09,True,NaN,8,306,44,1585
33,everyone sucks,aita refused to go in to see a movie after tic...,now this happened a while ago and i'm pretty s...,29.0,http://www.reddit.com/r/AmItheAsshole/comments...,31.0,1.423225e+09,True,NaN,14,415,65,2003
34,asshole,aita for refusing to not touch my phone as the...,i figured out during a 7 hour road trip to go ...,10.0,http://www.reddit.com/r/AmItheAsshole/comments...,30.0,1.423260e+09,True,NaN,18,177,99,952


In [30]:
data = aita[aita.flair.isin(['asshole', 'not the a-hole'])].head(2000)

data = data.dropna(subset=['text', 'flair', 'score'])

X = data.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,12]]
y = data.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



In [31]:


vectorizer = TfidfVectorizer(stop_words = 'english')

aiti_selftext_bgw = vectorizer.fit_transform(X_train.text)
aiti_selftext_bgw_test = vectorizer.transform(X_test.text)

# Convert bow_matrix into a DataFrame
bow_df = pd.DataFrame(aiti_selftext_bgw.toarray())
X_test = pd.DataFrame(aiti_selftext_bgw_test.toarray())

# Map the column names to vocabulary 
bow_df.columns = vectorizer.get_feature_names()
X_test.columns = vectorizer.get_feature_names()


In [83]:
aita.isna().sum()

flair                  0
title                  0
text                 186
score                  1
url                    1
num_comments           1
created_utc            1
is_self                1
num_crossposts    347465
title_wc               0
selftext_wc            0
title_cc               0
selftext_cc            0
dtype: int64

In [97]:
np.unique(pd.DataFrame(y_train), return_counts=True)

(array(['asshole', 'not the a-hole'], dtype=object),
 array([1672, 4988], dtype=int64))

In [99]:


# Fit the classifier
nb.fit(bow_df, y_train)

# Measure the accuracy
accuracy = nb.score(X_test, y_test)
print("The accuracy of the classifier on the test set is %.3f" % accuracy)

# Predict the sentiment of a negative review
post = "I tell you that I am confirmed asshole"
prediction = nb.predict(vectorizer.transform([post]))[0]

The accuracy of the classifier on the test set is 0.741


In [100]:
prediction

'not the a-hole'

In [105]:
preds = nb.predict(X_test)

In [106]:
np.unique(preds)

array(['not the a-hole'], dtype='<U14')

# SVC fitting

In [33]:
sv = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')

In [ ]:


# Fit the classifier
sv.fit(bow_df, y_train)

# Measure the accuracy
accuracy = sv.score(X_test, y_test)
print("The accuracy of the classifier on the test set is %.3f" % accuracy)

# Predict the sentiment of a negative review
post = ["I tell you that I am confirmed asshole", "Even my cat told me I am asshole. What do you think about this information?"]
prediction = sv.predict(vectorizer.transform([post]))[0]

In [28]:
preds = sv.predict(X_test)

In [29]:
np.unique(preds, return_counts=True)

(array(['asshole', 'not the a-hole'], dtype=object),
 array([  5, 321], dtype=int64))

# NB - it predicts only not the a-hole (the biggest class), SVC predicts outlying assholes (but still not enough)

We need to **reduce dimensions** (and take into account titles) - pure tfidf will not work

Maybe we should 
* create parts of speech features 
* delete words that occured in each class with the same frequency
* do sentiment analysis